# Подготовка пар предложений для [машинного перевода](08.Machine-Translation.LSTM-Seq2Seq-Attention.ipynb)

Русские и английские предложения взяты из раздела `Sentences` на сайте [tatoeba.org](https://tatoeba.org/en/downloads).

Также из раздела `Links` скачан файл, содержащий ссылки между предложениями.

Эти файлы я загрузил в свой Google Disk.

Предложения нужно подготовить:
- убрать дубликаты;
- минимальная длина предложения 3 токена;
- максимальная длина предложения 40 токенов;
- отобрать только 100k пар;
- Привести к формату `предложение_на_русском<seq>перевод_на_английском`. Каждая пара с новой строки;
- Сохранить в файл `rus-eng.txt`

In [1]:
import pandas as pd
import tokenizers
from tokenizers import pre_tokenizers

### Скачивание и разархивация

**Расскоментировать по необходимости**

In [2]:
# ru
# !gdown -c 1rtfpPatXiiN3lEfKtmxysvaQNlwiPadh -O rus_sentences.tsv.bz2

In [3]:
# en
# !gdown -c 1JRiXy0-AaaK-lYmeNmvXjVzDOXwbvuvk -O eng_sentences.tsv.bz2

In [4]:
# links
# !gdown -c 1Z6KqjjAg4EACmoHSjL97DLE9pVzDR3Tz -O links.tar.bz2

In [5]:
# !bunzip2 -k rus_sentences.tsv.bz2

In [6]:
# !bunzip2 -k eng_sentences.tsv.bz2

In [7]:
# !tar -xvjf links.tar.bz2

## Merge

Создание пар русский-английский в pandas.

In [8]:
rus_sentences = pd.read_csv(
    "rus_sentences.tsv",
    sep="\t",
    encoding="utf-8",
    names=["id", "lang", "sentence"],
)


In [9]:
eng_sentences = pd.read_csv(
    "eng_sentences.tsv",
    sep="\t",
    encoding="utf-8",
    names=["id", "lang", "sentence"],
)


In [10]:
links = pd.read_csv(
    "links.csv", sep="\t", encoding="utf-8", names=["lang1", "lang2"]
)

In [11]:
rus_sentences.head()

,id,lang,sentence
0,243,rus,Один раз в жизни я делаю хорошее дело... И оно...
1,5409,rus,Давайте что-нибудь попробуем!
2,5410,rus,Мне пора идти спать.
3,5411,rus,Что ты делаешь?
4,5412,rus,Что это?


In [12]:
eng_sentences.head()

,id,lang,sentence
0,1276,eng,Let's try something.
1,1277,eng,I have to go to sleep.
2,1280,eng,Today is June 18th and it is Muiriel's birthday!
3,1282,eng,Muiriel is 20 now.
4,1283,eng,"The password is ""Muiriel""."


In [13]:
links.head()

,lang1,lang2
0,1,2481
1,1,5350
2,1,180624
3,1,344899
4,1,380381


Оставляем только пары

In [14]:
rus_eng = links.merge(rus_sentences, left_on="lang1", right_on="id").merge(
    eng_sentences, left_on="lang2", right_on="id", suffixes=("_rus", "_eng")
)

In [15]:
rus_eng = rus_eng[["sentence_rus", "sentence_eng"]]

In [16]:
print(f"{rus_eng.shape=}")
rus_eng.head()

rus_eng.shape=(776798, 2)


,sentence_rus,sentence_eng
0,Один раз в жизни я делаю хорошее дело... И оно...,For once in my life I'm doing a good deed... A...
1,Давайте что-нибудь попробуем!,Let's try something.
2,Мне пора идти спать.,I have to go to sleep.
3,Мне пора идти спать.,It's time for me to go to bed.
4,Что ты делаешь?,What are you doing?


Был один дубликат

In [17]:
rus_eng.drop_duplicates(inplace=True, ignore_index=True)

## Фильтрация

- Воспользуемся только обычным предтокенизатором.

In [18]:
tokenizer = pre_tokenizers.Whitespace()

rus_eng = rus_eng.loc[
    rus_eng["sentence_rus"].apply(
        lambda x: 2 < len(tokenizer.pre_tokenize_str(x)) < 40
    )
].copy()

## Выборка

In [19]:
rus_eng = rus_eng.sample(100_000, random_state=42)

## Сохранение

In [20]:
lines = rus_eng["sentence_rus"] + "[SEP]" + rus_eng["sentence_eng"]

In [21]:
with open("rus_eng.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(lines))